In [1]:
import warnings  

import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
from torch.nn.functional import softmax

from efficientnet_pytorch import EfficientNet

from typing import Callable, List, Tuple, Dict
from pathlib import Path

from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict, OrderedDict
from tqdm.notebook import tqdm
from torchsummary import summary

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [2]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
EXPERIMENT_NAME = "01_efficientnet-b4"

class ConfigExperiment:
    logdir = f"./logs/{EXPERIMENT_NAME}"
    save_dirname = EXPERIMENT_NAME
    submission_file = f"{EXPERIMENT_NAME}.csv"
    seed = 42
    batch_size = 8
    model_name = 'efficientnet-b4'
    size = 512
    num_workers = 20
    root_images = "../../../data/raw/plant-pathology-2020-fgvc7/images/"
    root = "../../../data/raw/plant-pathology-2020-fgvc7/"
    num_classes = 4
    patience = 10
    early_stopping_delta = 1e-4
    num_epochs = 200
    lr = 0.003
    class_names = ["healthy", "multiple_diseases", "rust", "scab"]
    is_fp16_used = False
    
    
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
config = ConfigExperiment()
set_seed(config.seed)
config.size = EfficientNet.get_image_size(config.model_name)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Create target Directory
    os.mkdir(config.save_dirname)
    print("Directory " , config.save_dirname ,  " Created ") 
except FileExistsError:
    print("Directory " , config.save_dirname ,  " already exists")

Directory  01_efficientnet-b4  Created 


In [4]:
class PlantDataset(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.iloc[idx][self.class_names].values.astype(np.int8)
        label = torch.argmax(torch.from_numpy(labels))
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, label

In [5]:
def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        A.LongestMaxSize(max_size=image_size),
        A.PadIfNeeded(image_size, image_size, border_mode=0)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
#         A.RandomResizedCrop(height=config.size, width=config.size, p=1.0),
        A.OneOf([
            A.Rotate(limit=90, p=1),
            A.HorizontalFlip(p=1),
            A.VerticalFlip(p=1),
            A.OpticalDistortion(p=1),
            A.ShiftScaleRotate(rotate_limit=90, shift_limit=0.2, scale_limit=0.1, p=1)
        ], p=0.8),
        
        # Pixels
        A.OneOf([
            A.IAAEmboss(p=1.0),
            A.IAASharpen(p=1.0),
            A.Blur(p=1.0),
        ], p=0.5),

        # Affine
        A.OneOf([
            A.RandomContrast(limit=0.2, p=1),
            A.ElasticTransform(p=1),
            A.IAAPiecewiseAffine(p=1)
        ], p=0.5),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [A.Normalize(p=1.0), ToTensorV2(p=1.0),]

def compose(transforms_to_compose):
    # combine all augmentations into one single pipeline
    result = A.Compose([item for sublist in transforms_to_compose for item in sublist])
    return result

In [6]:
train_df = pd.read_csv(config.root + 'train.csv')
train, valid = train_test_split(train_df, test_size=0.33, random_state=config.seed, shuffle=True, stratify=train_df[config.class_names])

train_transforms = compose([
    pre_transforms(config.size),
    hard_transforms(), 
    post_transforms()
])
valid_transforms = compose([
    pre_transforms(config.size), 
    post_transforms()
])

show_transforms = compose([
    pre_transforms(config.size),
    hard_transforms()
])

train_dataset = PlantDataset(train, config, train_transforms)
valid_dataset = PlantDataset(valid, config, valid_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [7]:
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet") -> EfficientNet:
    model = EfficientNet.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(num_ftrs, num_classes, bias = True))
    return model

model = get_model(config.model_name, config.num_classes)

Loaded pretrained weights for efficientnet-b4


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True, mode="min", factor=0.3)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

# end_lr = 0.1
# start_lr = 1e-7
# lr_lambda = lambda x: math.exp(x * math.log(end_lr / start_lr) / (lr_find_epochs * len(train_dataloader)))
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# scheduler_1 = LinearCyclicalScheduler(optimizer, "lr", start_value=0.01, end_value=0.005, cycle_size=60)
# scheduler_2 = CosineAnnealingScheduler(optimizer, "lr", start_value=0.05, end_value=0.001, cycle_size=60)
# combined_scheduler = ConcatScheduler(schedulers=[scheduler_1, scheduler_2], durations=[30, ])

In [9]:
class Trainer:
    def __init__(self, model, train_dataloader: DataLoader, valid_dataloader: DataLoader, criterion, optimizer, scheduler, device, config: ConfigExperiment):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.config = config
        self.train_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.valid_metrics = {
            'avg_loss': [],
            'auc/_mean': [],
            'auc/healthy': [],
            'auc/multiple_diseases': [],
            'auc/rust': [],
            'auc/scab': [],
        }
        self.counter = 0
        self.delta = config.early_stopping_delta
      
    def run(self):
        self.model.to(device)
        best_valid_loss = float('inf')
        best_valid_auc_mean = 0

        try:
            for i_epoch in tqdm(range(self.config.num_epochs), desc='Epochs', total=config.num_epochs, position=1, leave=True):
                start_time = time.time()

                train_loss, train_outputs, train_targets = self._train()
                valid_loss, valid_outputs, valid_targets = self._evaluate()
                    
                self.train_metrics["avg_loss"].append(train_loss)
                self.train_metrics["auc/_mean"].append(self.comp_metric(train_outputs, train_targets))
                self.train_metrics["auc/healthy"].append(self.healthy_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/rust"].append(self.rust_roc_auc(train_outputs, train_targets))
                self.train_metrics["auc/scab"].append(self.scab_roc_auc(train_outputs, train_targets))
                
                self.valid_metrics["avg_loss"].append(valid_loss)
                self.valid_metrics["auc/_mean"].append(self.comp_metric(valid_outputs, valid_targets))
                self.valid_metrics["auc/healthy"].append(self.healthy_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/multiple_diseases"].append(self.multiple_diseases_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/rust"].append(self.rust_roc_auc(valid_outputs, valid_targets))
                self.valid_metrics["auc/scab"].append(self.scab_roc_auc(valid_outputs, valid_targets))
                
                end_time = time.time()
                epoch_mins, epoch_secs = self._epoch_time(start_time, end_time)
                self.print_progress(i_epoch, epoch_mins, epoch_secs)
                
                self.scheduler.step(self.valid_metrics["auc/_mean"][-1])
                
                if valid_loss < best_valid_loss:
                    self.counter = 0
                    best_valid_loss = valid_loss
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_loss={best_valid_loss}.pth")
                else:
                    self.counter += 1
                    
                if self.valid_metrics["auc/_mean"][-1] > best_valid_auc_mean:
                    best_valid_auc_mean = self.valid_metrics["auc/_mean"][-1]
                    torch.save(model.state_dict(), f"{config.save_dirname}/best_model_epoch={i_epoch+1}_auc_mean={best_valid_auc_mean}.pth")
                    
                if self.counter > self.config.patience:
                    print("EarlyStopping")
                    break
        except KeyboardInterrupt: 
            pass
        
        return self.train_metrics, self.valid_metrics
        
    def _train(self):
        model.train()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        for i, (images, labels) in tqdm(enumerate(self.train_dataloader), desc='Train', total=len(self.train_dataloader), position=2, leave=True):
            loss_iten, outputs = self._train_process(images, labels)
            epoch_loss += loss_iten              

            if epoch_output is None:
                epoch_output = outputs.cpu().data
            else:
                epoch_output = torch.cat((epoch_output, outputs.cpu().data))

            if epoch_target is None:
                epoch_target = labels.cpu().data
            else:
                epoch_target = torch.cat((epoch_target, labels.cpu().data))
            
        return epoch_loss / len(self.train_dataloader), epoch_output, epoch_target
    
    def _train_process(self, images, labels):
        images = images.to(self.device)
        labels = labels.to(self.device)
        self.optimizer.zero_grad()
        outputs = self.model(images)
        loss = self.criterion(outputs, labels)
        loss.backward()
        self.optimizer.step()
        return loss.item(), outputs
            
    def _evaluate(self):
        model.eval()
        epoch_loss = 0
        epoch_output = None
        epoch_target = None
        with torch.no_grad():
            for i, (images, labels) in enumerate(self.valid_dataloader):
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                epoch_loss += loss.item()
                
                if epoch_output is None:
                    epoch_output = outputs.cpu().data
                else:
                    epoch_output = torch.cat((epoch_output, outputs.cpu().data))

                if epoch_target is None:
                    epoch_target = labels.cpu().data
                else:
                    epoch_target = torch.cat((epoch_target, labels.cpu().data))

        return epoch_loss / len(self.valid_dataloader), epoch_output, epoch_target
    
#     def _evaluate_tta(self):
#         n_samples = len(dataset)
#         y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)

#         model.eval()
#         model.to(device)
#         with torch.no_grad():
#             for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
#                 for i, (images, idx) in enumerate(dataloader):
#                     images = images.to(device)
#                     outputs = model(images)
#                     y_pred = F.softmax(outputs, dim=1)

#                     tta_index = i_epoch - 1
#                     start_index = (i % len(dataloader)) * config.batch_size
#                     end_index = min(start_index + config.batch_size, n_samples)
#                     batch_y_probas = y_pred.detach().cpu().numpy()                
#                     y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

#         return y_probas_tta

#     def show_history(self):
#         fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
#         clear_output(True)
#         ax[0].plot(history, label='train loss')
#         ax[0].set_xlabel('Batch')
#         ax[0].set_title('Train loss')
#         if train_history is not None:
#             ax[1].plot(train_history, label='general train history')
#             ax[1].set_xlabel('Epoch')
#         if valid_history is not None:
#             ax[1].plot(valid_history, label='general valid history')
#         plt.legend()
#         plt.show()

    
    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def print_progress(self, i_epoch, epoch_mins, epoch_secs):
        i_epoch = i_epoch + 1
        print(f"Epoch: {i_epoch:02} | Time: {epoch_mins}m {epoch_secs}s")
        print("Training Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format(
                self.train_metrics['avg_loss'][-1], 
                self.train_metrics['auc/_mean'][-1],
                self.train_metrics['auc/healthy'][-1],
                self.train_metrics['auc/multiple_diseases'][-1],
                self.train_metrics['auc/rust'][-1],
                self.train_metrics['auc/scab'][-1],
            ))
        print("Evaluating Results - Average Loss: {:.4f} | auc/_mean: {:.4f} | auc/healthy: {:.4f} | auc/multiple_diseases: {:.4f}, auc/rust: {:.4f}, auc/scab: {:.4f}"
            .format( 
                self.valid_metrics['avg_loss'][-1], 
                self.valid_metrics['auc/_mean'][-1],
                self.valid_metrics['auc/healthy'][-1],
                self.valid_metrics['auc/multiple_diseases'][-1],
                self.valid_metrics['auc/rust'][-1],
                self.valid_metrics['auc/scab'][-1],
            ))
        print()
        
    def comp_metric(self, preds, targs, labels=range(4)):
        preds = torch.sigmoid(preds)
        targs = torch.eye(4)[targs]
        return np.mean([roc_auc_score(targs[:,i], preds[:,i]) for i in labels])

    def healthy_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[0])

    def multiple_diseases_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[1])

    def rust_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[2])

    def scab_roc_auc(self, *args):
        return self.comp_metric(*args, labels=[3])



In [10]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 0m 34s
Training Results - Average Loss: 0.9242 | auc/_mean: 0.7632 | auc/healthy: 0.8168 | auc/multiple_diseases: 0.5830, auc/rust: 0.8342, auc/scab: 0.8187
Evaluating Results - Average Loss: 0.5458 | auc/_mean: 0.8881 | auc/healthy: 0.9550 | auc/multiple_diseases: 0.6667, auc/rust: 0.9828, auc/scab: 0.9480




Epoch: 02 | Time: 0m 34s
Training Results - Average Loss: 0.7900 | auc/_mean: 0.8196 | auc/healthy: 0.8818 | auc/multiple_diseases: 0.6824, auc/rust: 0.8712, auc/scab: 0.8431
Evaluating Results - Average Loss: 0.4228 | auc/_mean: 0.9170 | auc/healthy: 0.9682 | auc/multiple_diseases: 0.7621, auc/rust: 0.9786, auc/scab: 0.9591




Epoch: 03 | Time: 0m 34s
Training Results - Average Loss: 0.7192 | auc/_mean: 0.8428 | auc/healthy: 0.8845 | auc/multiple_diseases: 0.7168, auc/rust: 0.8940, auc/scab: 0.8760
Evaluating Results - Average Loss: 0.4017 | auc/_mean: 0.9203 | auc/healthy: 0.9723 | auc/multiple_diseases: 0.7736, auc/rust: 0.9750, auc/scab: 0.9602




Epoch: 04 | Time: 0m 34s
Training Results - Average Loss: 0.7035 | auc/_mean: 0.8493 | auc/healthy: 0.8747 | auc/multiple_diseases: 0.7307, auc/rust: 0.8974, auc/scab: 0.8943
Evaluating Results - Average Loss: 0.3858 | auc/_mean: 0.9187 | auc/healthy: 0.9718 | auc/multiple_diseases: 0.7689, auc/rust: 0.9766, auc/scab: 0.9574




Epoch: 05 | Time: 0m 34s
Training Results - Average Loss: 0.6825 | auc/_mean: 0.8523 | auc/healthy: 0.8942 | auc/multiple_diseases: 0.7149, auc/rust: 0.9099, auc/scab: 0.8899
Evaluating Results - Average Loss: 0.3766 | auc/_mean: 0.9211 | auc/healthy: 0.9746 | auc/multiple_diseases: 0.7738, auc/rust: 0.9758, auc/scab: 0.9603

Epoch     5: reducing learning rate of group 0 to 9.0000e-04.


...


Epoch: 20 | Time: 0m 34s
Training Results - Average Loss: 0.6464 | auc/_mean: 0.8763 | auc/healthy: 0.8990 | auc/multiple_diseases: 0.8019, auc/rust: 0.9161, auc/scab: 0.8882
Evaluating Results - Average Loss: 0.3515 | auc/_mean: 0.9262 | auc/healthy: 0.9733 | auc/multiple_diseases: 0.7855, auc/rust: 0.9790, auc/scab: 0.9671




Epoch: 21 | Time: 0m 34s
Training Results - Average Loss: 0.6374 | auc/_mean: 0.8761 | auc/healthy: 0.9097 | auc/multiple_diseases: 0.7875, auc/rust: 0.9031, auc/scab: 0.9040
Evaluating Results - Average Loss: 0.3524 | auc/_mean: 0.9261 | auc/healthy: 0.9728 | auc/multiple_diseases: 0.7858, auc/rust: 0.9788, auc/scab: 0.9669

Epoch    21: reducing learning rate of group 0 to 7.2900e-06.



Epoch: 22 | Time: 0m 34s
Training Results - Average Loss: 0.6003 | auc/_mean: 0.8770 | auc/healthy: 0.9111 | auc/multiple_diseases: 0.7692, auc/rust: 0.9204, auc/scab: 0.9074
Evaluating Results - Average Loss: 0.3517 | auc/_mean: 0.9268 | auc/healthy: 0.9729 | auc/multiple_diseases: 0.7891, auc/rust: 0.9786, auc/scab: 0.9664




Epoch: 23 | Time: 0m 34s
Training Results - Average Loss: 0.6544 | auc/_mean: 0.8674 | auc/healthy: 0.8924 | auc/multiple_diseases: 0.7686, auc/rust: 0.9062, auc/scab: 0.9024
Evaluating Results - Average Loss: 0.3530 | auc/_mean: 0.9265 | auc/healthy: 0.9726 | auc/multiple_diseases: 0.7871, auc/rust: 0.9788, auc/scab: 0.9674




Epoch: 24 | Time: 0m 34s
Training Results - Average Loss: 0.6001 | auc/_mean: 0.8759 | auc/healthy: 0.9001 | auc/multiple_diseases: 0.7669, auc/rust: 0.9215, auc/scab: 0.9151
Evaluating Results - Average Loss: 0.3529 | auc/_mean: 0.9264 | auc/healthy: 0.9726 | auc/multiple_diseases: 0.7871, auc/rust: 0.9785, auc/scab: 0.9675

EarlyStopping


In [13]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=13_loss=0.3506331098216929.pth"))
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=config.patience, verbose=True, mode="min", factor=0.3)

Loaded pretrained weights for efficientnet-b4


In [14]:
trainer = Trainer(model, train_dataloader, valid_dataloader, criterion, optimizer, scheduler, device, config)
trainer.run();


Epoch: 01 | Time: 1m 30s
Training Results - Average Loss: 1.2435 | auc/_mean: 0.7705 | auc/healthy: 0.7919 | auc/multiple_diseases: 0.6151, auc/rust: 0.8696, auc/scab: 0.8056
Evaluating Results - Average Loss: 1.8008 | auc/_mean: 0.8325 | auc/healthy: 0.9320 | auc/multiple_diseases: 0.6655, auc/rust: 0.7981, auc/scab: 0.9345




Epoch: 02 | Time: 1m 29s
Training Results - Average Loss: 0.6471 | auc/_mean: 0.8404 | auc/healthy: 0.9153 | auc/multiple_diseases: 0.5916, auc/rust: 0.9502, auc/scab: 0.9044
Evaluating Results - Average Loss: 0.2592 | auc/_mean: 0.8941 | auc/healthy: 0.9941 | auc/multiple_diseases: 0.5970, auc/rust: 0.9963, auc/scab: 0.9890




Epoch: 03 | Time: 1m 29s
Training Results - Average Loss: 0.4973 | auc/_mean: 0.8587 | auc/healthy: 0.9576 | auc/multiple_diseases: 0.5665, auc/rust: 0.9734, auc/scab: 0.9372
Evaluating Results - Average Loss: 0.3834 | auc/_mean: 0.8689 | auc/healthy: 0.9789 | auc/multiple_diseases: 0.5172, auc/rust: 0.9942, auc/scab: 0.9855




Epoch: 04 | Time: 1m 29s
Training Results - Average Loss: 0.4243 | auc/_mean: 0.8794 | auc/healthy: 0.9736 | auc/multiple_diseases: 0.6103, auc/rust: 0.9712, auc/scab: 0.9627
Evaluating Results - Average Loss: 0.7639 | auc/_mean: 0.8930 | auc/healthy: 0.9912 | auc/multiple_diseases: 0.6107, auc/rust: 0.9824, auc/scab: 0.9877




Epoch: 05 | Time: 1m 29s
Training Results - Average Loss: 0.4526 | auc/_mean: 0.8714 | auc/healthy: 0.9738 | auc/multiple_diseases: 0.5886, auc/rust: 0.9635, auc/scab: 0.9597
Evaluating Results - Average Loss: 0.3116 | auc/_mean: 0.8842 | auc/healthy: 0.9925 | auc/multiple_diseases: 0.5605, auc/rust: 0.9959, auc/scab: 0.9876



...


Epoch: 32 | Time: 1m 30s
Training Results - Average Loss: 0.0889 | auc/_mean: 0.9726 | auc/healthy: 0.9992 | auc/multiple_diseases: 0.8958, auc/rust: 0.9966, auc/scab: 0.9988
Evaluating Results - Average Loss: 0.1318 | auc/_mean: 0.9777 | auc/healthy: 0.9991 | auc/multiple_diseases: 0.9171, auc/rust: 0.9986, auc/scab: 0.9959




Epoch: 33 | Time: 1m 32s
Training Results - Average Loss: 0.0768 | auc/_mean: 0.9768 | auc/healthy: 0.9997 | auc/multiple_diseases: 0.9116, auc/rust: 0.9975, auc/scab: 0.9985
Evaluating Results - Average Loss: 0.1568 | auc/_mean: 0.9610 | auc/healthy: 0.9991 | auc/multiple_diseases: 0.8504, auc/rust: 0.9985, auc/scab: 0.9961




Epoch: 34 | Time: 1m 30s
Training Results - Average Loss: 0.0819 | auc/_mean: 0.9744 | auc/healthy: 0.9975 | auc/multiple_diseases: 0.9057, auc/rust: 0.9972, auc/scab: 0.9974
Evaluating Results - Average Loss: 0.1716 | auc/_mean: 0.9685 | auc/healthy: 0.9972 | auc/multiple_diseases: 0.8813, auc/rust: 0.9985, auc/scab: 0.9971

Epoch    34: reducing learning rate of group 0 to 8.1000e-05.



Epoch: 35 | Time: 1m 30s
Training Results - Average Loss: 0.0598 | auc/_mean: 0.9842 | auc/healthy: 0.9995 | auc/multiple_diseases: 0.9395, auc/rust: 0.9986, auc/scab: 0.9994
Evaluating Results - Average Loss: 0.1655 | auc/_mean: 0.9649 | auc/healthy: 0.9986 | auc/multiple_diseases: 0.8654, auc/rust: 0.9986, auc/scab: 0.9970




Epoch: 36 | Time: 1m 30s
Training Results - Average Loss: 0.0614 | auc/_mean: 0.9802 | auc/healthy: 0.9997 | auc/multiple_diseases: 0.9239, auc/rust: 0.9989, auc/scab: 0.9981
Evaluating Results - Average Loss: 0.1541 | auc/_mean: 0.9619 | auc/healthy: 0.9985 | auc/multiple_diseases: 0.8531, auc/rust: 0.9985, auc/scab: 0.9973

EarlyStopping


In [23]:

class PlantDatasetTest(Dataset):
    
    def __init__(self, df, config, transforms=None):
    
        self.df = df
        self.images_dir = config.root_images
        self.class_names = config.class_names
        self.transforms=transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_src = self.images_dir + self.df.iloc[idx]['image_id'] + '.jpg'
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image, idx

test_df = pd.read_csv(config.root + 'test.csv')
test_dataset = PlantDatasetTest(test_df, config, train_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=config.num_workers)

In [24]:
def predict(model, dataloader: DataLoader, device, dataset: Dataset, tta_count: int, config: ConfigExperiment):
    n_samples = len(dataset)
    y_probas_tta = np.zeros((n_samples, config.num_classes, tta_count), dtype=np.float32)
    
    model.eval()
    model.to(device)
    with torch.no_grad():
        for i_epoch in tqdm(range(tta_count), desc='TTA', total=tta_count, position=1, leave=True):
            for i, (images, idx) in enumerate(dataloader):
                images = images.to(device)
                outputs = model(images)
                y_pred = F.softmax(outputs, dim=1)
                
                tta_index = i_epoch - 1
                start_index = (i % len(dataloader)) * config.batch_size
                end_index = min(start_index + config.batch_size, n_samples)
                batch_y_probas = y_pred.detach().cpu().numpy()                
                y_probas_tta[start_index:end_index, :, tta_index] = batch_y_probas

    return y_probas_tta

In [25]:
model = get_model(config.model_name, config.num_classes)
model.load_state_dict(torch.load(f"{config.save_dirname}/best_model_epoch=32_auc_mean=0.9776760373798699.pth"))

Loaded pretrained weights for efficientnet-b4


<All keys matched successfully>

In [26]:
result = predict(model, test_dataloader, device, test_dataset, tta_count=5, config=config)

In [27]:
y_probas = np.mean(result, axis=-1)

In [28]:
y_probas[:10]

array([[6.3880638e-04, 6.9609387e-03, 9.9238980e-01, 1.0497751e-05],
       [1.7404766e-04, 8.8852663e-03, 9.9091434e-01, 2.6263195e-05],
       [1.1449547e-02, 4.1036732e-03, 3.0086001e-05, 9.8441678e-01],
       [9.9822557e-01, 2.0357803e-04, 5.0409049e-06, 1.5657895e-03],
       [3.0939223e-03, 2.4749709e-03, 9.9442291e-01, 8.2185443e-06],
       [9.8253727e-01, 1.4549071e-02, 1.6253630e-03, 1.2883531e-03],
       [9.9889326e-01, 9.0801048e-05, 6.8565423e-06, 1.0090604e-03],
       [1.8453782e-06, 4.8701063e-02, 3.5145411e-05, 9.5126200e-01],
       [1.7212449e-03, 1.6756514e-01, 2.1020518e-04, 8.3050334e-01],
       [2.2361132e-04, 1.9035389e-03, 9.9786985e-01, 3.0038955e-06]],
      dtype=float32)

In [29]:
test_df = pd.read_csv(config.root + 'test.csv')
test_df["healthy"] = 0
test_df["multiple_diseases"] = 0
test_df["rust"] = 0
test_df["scab"] = 0
test_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = y_probas
test_df.to_csv(config.submission_file, index=False)
test_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.000639,0.006961,0.992390,0.000010
1,Test_1,0.000174,0.008885,0.990914,0.000026
2,Test_2,0.011450,0.004104,0.000030,0.984417
3,Test_3,0.998226,0.000204,0.000005,0.001566
4,Test_4,0.003094,0.002475,0.994423,0.000008


In [30]:
config.submission_file

'01_efficientnet-b4.csv'